In [1]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision

# To view tensorboard metrics
# tensorboard --logdir=logs --port=6006 --bind_all
from torch.utils.tensorboard import SummaryWriter
from functools import partial
from evolver import CrossoverType, MutationType, VectorEvolver, InitType
from unet import UNet
from dataset_utils import PartitionType
from cuda_utils import maybe_get_cuda_device, clear_cuda
from landcover_dataloader import get_landcover_dataloaders

from ignite.contrib.handlers.tensorboard_logger import *
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, ConfusionMatrix, mIoU
from ignite.handlers import ModelCheckpoint
from ignite.utils import setup_logger
from ignite.engine import Engine
import sys

# Define directories for data, logging and model saving.
base_dir = os.getcwd()
dataset_name = "landcover_large"
dataset_dir = os.path.join(base_dir, "data/" + dataset_name)

experiment_name = "backprop_single_point_finetuning_test_test"
model_name = "best_model_30_validation_accuracy=0.9409.pt"
model_path = os.path.join(base_dir, "logs/" + dataset_name + "/" + model_name)
log_dir = os.path.join(base_dir, "logs/" + dataset_name + "_" + experiment_name)

# Create DataLoaders for each partition of Landcover data.
dataloader_params = {
    'batch_size': 1,
    'shuffle': True,
    'num_workers': 6,
    'pin_memory': True}

partition_types = [PartitionType.TRAIN, PartitionType.VALIDATION, 
                   PartitionType.FINETUNING, PartitionType.TEST]
data_loaders = get_landcover_dataloaders(dataset_dir, 
                                         partition_types,
                                         dataloader_params,
                                         force_create_dataset=True)

finetuning_loader = data_loaders[2]
test_loader = data_loaders[3]

# Get GPU device if available.
device = maybe_get_cuda_device()

# Determine model and training params.
params = {
    'max_epochs': 10,
    'n_classes': 4,
    'in_channels': 4,
    'depth': 5,
    'learning_rate': 0.01,
    'log_steps': 1,
    'save_top_n_models': 4
}

clear_cuda()    
model = UNet(in_channels = params['in_channels'],
             n_classes = params['n_classes'],
             depth = params['depth'])
model.load_state_dict(torch.load(model_path))

model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=params['learning_rate'])


# Determine metrics for evaluation.
train_metrics = {
        "accuracy": Accuracy(), 
        "loss": Loss(criterion),
        "mean_iou": mIoU(ConfusionMatrix(num_classes = params['n_classes'])),
        }

validation_metrics = {
        "accuracy": Accuracy(), 
        "loss": Loss(criterion),
        "mean_iou": mIoU(ConfusionMatrix(num_classes = params['n_classes'])),

}

import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
for batch in finetuning_loader:
    batch_x = batch[0]
    _ = model(batch_x)
    break
    
drop_out_layers = model.get_dropout_layers()


    
def mask_from_vec(vec, matrix_size):
    mask = np.ones(matrix_size)
    for i in range(len(vec)):
        if vec[i] == 0:
            mask[i, :, :] = 0

        elif vec[i] == 1:
            mask[i, :, :] = 1

    return mask



for layer in drop_out_layers:
    layer_name = layer.name
    size = layer.x_size[1:]
    sizes = [size]
    clear_cuda()    
    model = UNet(in_channels = params['in_channels'],
                 n_classes = params['n_classes'],
                 depth = params['depth'])
    model.load_state_dict(torch.load(model_path))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=params['learning_rate'])

    num_channels = size[0]
    evolver = VectorEvolver(num_channels, 
                            CrossoverType.UNIFORM,
                            MutationType.FLIP_BIT, 
                            InitType.BINOMIAL, 
                            flip_bit_prob=None, 
                            flip_bit_decay=1.0,
                            binomial_prob=0.8)

    print("LAYER", layer_name, size)
    with torch.no_grad():
        batch_x, batch_y = batch
        loss = sys.float_info.max
        child_mask_prev = None
        for i in range(5):
            child_vec = evolver.spawn_child()
            child_mask = mask_from_vec(child_vec, size)
            model.set_dropout_masks({layer_name: torch.tensor(child_mask, dtype=torch.float32)})
            outputs = model(batch_x)
            current_loss = criterion(outputs[:, :, 127:128,127:128], batch_y[:,127:128,127:128]).item()
            evolver.add_child(child_mask, 1.0 / current_loss)
            print("Current", current_loss)
            loss = min(loss, current_loss)
            
#             f, ax = plt.subplots(1, 3, figsize=(20, 8))
#             ax[0].imshow((np.array(batch_y.detach().numpy()[0, :, :])))
#             ax[1].imshow(np.argmax(np.moveaxis(np.array(outputs.detach().numpy()[0, :, :, :]), [0],[ 2]), axis=2))
#             ax[2].imshow(child_mask[0, :, :])
              
#             child_mask_prev = child_mask
#             plt.show()
        print("best_loss", 1.0/evolver.get_best_child()[0])

            

LAYER start torch.Size([64, 256, 256])
Current 0.003959861118346453
Current 0.010542606003582478
Current 0.0416293665766716
Current 0.1551894247531891
Current 0.16211389005184174
best_loss 0.003959861118346453
LAYER down_0 torch.Size([128, 128, 128])
Current 3.989912748336792


KeyboardInterrupt: 

In [ ]:
evolver.get_best_child()